KeyboardInterrupt: 

In [9]:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from tqdm import tqdm

# Authenticate and create PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Specify the folder ID
folder_id = '1-V6h8LO7e7frqgq0XoAxMcJTeUNYFoVe'

# Fetch list of files in the folder
files = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder_id)}).GetList()

# Create a directory to store downloaded images
save_directory = "/content/images"
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

# Download each file and show progress bar
for file in tqdm(files, desc="Downloading files", unit="file"):
    file.GetContentFile(os.path.join(save_directory, file['title']))


In [10]:
import os
import cv2
from tqdm import tqdm

def resize_images(input_dir, output_dir):
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # List all files in the input directory
    files = os.listdir(input_dir)

    # Initialize tqdm for progress bar
    progress_bar = tqdm(total=len(files), desc="Resizing images", unit="image")

    # Resize each image and save to the output directory
    for file in files:
        try:
            # Open the image file
            img_path = os.path.join(input_dir, file)
            img = cv2.imread(img_path)

            # Resize the image to 1024x1024
            img_resized = cv2.resize(img, (1024, 1024), interpolation=cv2.INTER_AREA)

            # Save the resized image to the output directory
            output_path = os.path.join(output_dir, file)
            cv2.imwrite(output_path, img_resized)

            # Update progress bar
            progress_bar.update(1)
            progress_bar.set_postfix({"File": file})

        except Exception as e:
            print(f"Error processing {file}: {e}")

    # Close progress bar
    progress_bar.close()

# Example usage:
input_directory = "/content/images"
output_directory = "/content/images_resized"

resize_images(input_directory, output_directory)



Resizing images: 100%|██████████| 513/513 [03:19<00:00,  2.57image/s, File=IMG_7970.JPG]


In [11]:
import os
import shutil
import zipfile
from tqdm import tqdm

def zip_dir(directory, zip_name):
    # Create a zip file containing all files in the directory
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        files = os.listdir(directory)
        for file in tqdm(files, desc="Creating zip file", unit="file"):
            file_path = os.path.join(directory, file)
            zipf.write(file_path, arcname=file)

# Directory to download
directory_to_download = "/content/images_resized"

# Zip file name
zip_file_name = "images_resized.zip"

# Create the zip file with progress bar
zip_dir(directory_to_download, zip_file_name)

# Move the zip file to /content directory
shutil.move(zip_file_name, "/content")

# Provide the link to download the zip file
print("Download the zip file containing the resized images:")
print(f"/content/{zip_file_name}")



Creating zip file: 100%|██████████| 513/513 [00:17<00:00, 29.93file/s]


Error: Destination path '/content/images_resized.zip' already exists